# CS246 - Colab 7
## Decision Trees in Spark

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 49.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=4bda57ac2689e0c83265ae48e839b9b37947758a790addb6635ccc911ce4554e
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Rea

Now we authenticate a Google Drive client to download the files we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1aJrdYMVmmnUKYhLTlXtyB0FQ9gYJqCrs'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('mnist-digits-train.txt')

id='1yLwxRaJIyrC03yxqbTKpedMmHEF86AAq'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('mnist-digits-test.txt')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Data Loading

![MNIST](https://upload.wikimedia.org/wikipedia/commons/thumb/2/27/MnistExamples.png/220px-MnistExamples.png)

In this Colab, we will be using the famous [MNIST database](https://en.wikipedia.org/wiki/MNIST_database), a large collection of handwritten digits that is widely used for training and testing in the field of machine learning.

For your convenience, the dataset has already been converted to the popular LibSVM format, where each digit is represented as a sparse vector of grayscale pixel values.

In [6]:
training = spark.read.format("libsvm").load("mnist-digits-train.txt")
test = spark.read.format("libsvm").load("mnist-digits-test.txt")

# Cache data for multiple uses
training.cache()
test.cache()

DataFrame[label: double, features: vector]

In [7]:
training.show(truncate=False)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
training.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [9]:
test.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



### Your task

First of all, find out how many instances we have in our training / test split.

In [10]:
# YOUR CODE HERE
print(training)

DataFrame[label: double, features: vector]


Now train a Decision Tree on the training dataset using Spark MLlib.

You can refer to the Python example on this documentation page: [https://spark.apache.org/docs/latest/ml-classification-regression.html#decision-tree-classifier](https://spark.apache.org/docs/latest/ml-classification-regression.html#decision-tree-classifier)

(Note : You **don't** need to and should **NOT** use Indexer as shown by the example, so that your answers can match our solution answers for Gradescope problems.)

In [11]:
# YOUR CODE HERE
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")
model = dt.fit(training)

With the Decision Tree you just induced on the training data, predict the labels of the test set.
Print the predictions for the first 10 digits, and compare them with the labels.

In [14]:
# YOUR CODE HERE
predictions = model.transform(test)
predictions.show(10)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  7.0|(778,[202,203,204...|[131.0,71.0,318.0...|[0.02414301511242...|       7.0|
|  2.0|(778,[94,95,96,97...|[19.0,35.0,188.0,...|[0.03598484848484...|       6.0|
|  1.0|(778,[128,129,130...|[3.0,5553.0,77.0,...|[4.73260766682442...|       1.0|
|  0.0|(778,[124,125,126...|[4325.0,2.0,68.0,...|[0.92572773972602...|       0.0|
|  4.0|(778,[150,151,159...|[116.0,0.0,83.0,6...|[0.09048361934477...|       9.0|
|  1.0|(778,[156,157,158...|[3.0,5553.0,77.0,...|[4.73260766682442...|       1.0|
|  4.0|(778,[149,150,151...|[6.0,36.0,12.0,30...|[0.00473933649289...|       5.0|
|  9.0|(778,[179,180,181...|[13.0,1.0,5.0,94....|[0.01203703703703...|       9.0|
|  5.0|(778,[129,130,131...|[258.0,6.0,100.0,...|[0.09123055162659...|       6.0|
|  9.0|(778,[209

The small sample above looks good, but not great!

Let's dig deeper. Compute the accuracy of our model, using the ```MulticlassClassificationEvaluator``` from MLlib.

In [17]:
# YOUR CODE HERE
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Accuracy = %g " % (accuracy))

Test Accuracy = 0.6746 


Find out the max depth of the trained Decision Tree, and its total number of nodes.

In [23]:
# YOUR CODE HERE
# summary only
model.depth, model.numNodes

(5, 59)

It appears that the default settings of the Decision Tree implemented in MLlib did not allow us to train a very powerful model!

Before starting to train a Decision Tree, you can tune the max depth it can reach using the [setMaxDepth()](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.DecisionTreeClassifier.html#pyspark.ml.classification.DecisionTreeClassifier.setMaxDepth) method. Train 21 different DTs, varying the max depth from 0 to 20, endpoints included (i.e., [0, 20]). For each value of the parameter, print the accuracy achieved on the test set, and the number of nodes contained in the given DT.

For this problem, we say that the decision trees overfit the dataset if the classification accuracy starts to drop.

**IMPORTANT:** this parameter sweep can take 30 minutes or more, depending on how busy is your Colab instance. Notice how the induction time grows super-linearly!

In [24]:
# YOUR CODE HERE
for i in range(0, 21):
  print("now evaluating dt with maxdepth %d" %(i))
  dt = DecisionTreeClassifier(labelCol="label", featuresCol="features",maxDepth=i)
  model = dt.fit(training)
  predictions = model.transform(test)
  accuracy = evaluator.evaluate(predictions)
  print("Test Accuracy for dt with maxdepth %d = %g " % (i, accuracy))

now evaluating dt with maxdepth 0
Test Accuracy for dt with maxdepth 0 = 0.1135 
now evaluating dt with maxdepth 1
Test Accuracy for dt with maxdepth 1 = 0.1994 
now evaluating dt with maxdepth 2
Test Accuracy for dt with maxdepth 2 = 0.3447 
now evaluating dt with maxdepth 3
Test Accuracy for dt with maxdepth 3 = 0.4947 
now evaluating dt with maxdepth 4
Test Accuracy for dt with maxdepth 4 = 0.595 
now evaluating dt with maxdepth 5
Test Accuracy for dt with maxdepth 5 = 0.6746 
now evaluating dt with maxdepth 6
Test Accuracy for dt with maxdepth 6 = 0.742 
now evaluating dt with maxdepth 7
Test Accuracy for dt with maxdepth 7 = 0.785 
now evaluating dt with maxdepth 8
Test Accuracy for dt with maxdepth 8 = 0.8189 
now evaluating dt with maxdepth 9
Test Accuracy for dt with maxdepth 9 = 0.8512 
now evaluating dt with maxdepth 10
Test Accuracy for dt with maxdepth 10 = 0.8675 
now evaluating dt with maxdepth 11
Test Accuracy for dt with maxdepth 11 = 0.8756 
now evaluating dt with maxd

Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!